<a href="https://colab.research.google.com/github/subramanya4shenoy/MachineLearningNbs/blob/main/kaggle/playground_series_s4e9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload() # Upload your kaggle.json here

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q kaggle

In [4]:
!kaggle competitions download -c playground-series-s4e9

 64% 5.00M/7.84M [00:00<00:00, 51.6MB/s]
100% 7.84M/7.84M [00:00<00:00, 67.6MB/s]


In [5]:
!unzip "playground-series-s4e9"

Archive:  playground-series-s4e9.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# 🚙 Used car price predictor 🚙

In [6]:
# common imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 🟢 Reading the data

In [7]:
# reading the train and test data and saving it in train and test dataframes
train = pd.read_csv("train.csv", index_col='id')
test = pd.read_csv("test.csv", index_col='id')

## 🟢 Exploratory data analysis

In [ ]:
# insights from the train set
train.shape

(188533, 12)

In [ ]:
train.columns

Index(['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price'],
      dtype='object')

In [ ]:
train.dtypes.value_counts()

,count
object,9
int64,3


In [ ]:
train.head(100)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500
...,...,...,...,...,...,...,...,...,...,...,...,...
95,INFINITI,QX60 Base,2016,95400,Gasoline,265.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Beige,At least 1 accident or damage reported,Yes,15999
96,Dodge,Ram 1500 Laramie Mega Cab,2005,153692,Gasoline,345.0HP 5.7L 8 Cylinder Engine Gasoline Fuel,A/T,Blue,Gray,At least 1 accident or damage reported,Yes,8300
97,Ford,F-150 XLT,2018,63500,Gasoline,400.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,26500


### 🟡 Observation


*   There are 1,88,533 records to train
*   Theere are 12 columns
*   1 column (price) to predict, 11 attributes, 9 categorical column, 2 numeric columns
*   As there are lot of categorical columns we will be trying to clean up each columns and try to infer meaningfull data





In [11]:
# exploring each categorical columns
cat_cols = train.select_dtypes(include='object').columns
cat_cols

Index(['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col',
       'int_col', 'accident', 'clean_title'],
      dtype='object')

### ⭕ 'Brand' column

*   There are 57 uniq brands listed with Ford being in high frequency
*   categorical data can be replaced with numbers, using OrdinalEncoder from sklearn



In [8]:
# 'Brand' Columns
train['brand'].describe()

,brand
count,188533
unique,57
top,Ford
freq,23088


In [12]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()